In [1]:
# increase the cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; } </style>"))

# print table
import sys
sys.path.insert(-1, '/xdisk/msurdeanu/fanluo/miniconda3/lib/python3.7/site-packages') 
from prettytable import PrettyTable
from textwrap import fill

## <a href='https://github.com/recognai/spacy-wordnet' style='text-decoration:none'>spaCy WordNet</a>  <a class="tocSkip">
- <a href='https://wordnet.princeton.edu/' style='text-decoration:none'>WordNet</a>
    - A large lexical database of English.
    - 117 000 unordered sets of cognitive synonyms (synsets).
    - Four sub-nets (nouns, verbs, adjectives and adverbs), with a few cross-POS pointers. <br>
      For example: observe (verb), observant (adjective) observation, observatory (nouns)
  
- <a href='https://www.nltk.org/howto/wordnet.html' style='text-decoration:none'>NLTK WordNet Interface</a> <br>
- <a href='http://wndomains.fbk.eu/' style='text-decoration:none'>WordNet domains</a>

In [3]:
import spacy
import sys
sys.path.insert(-1, '/xdisk/msurdeanu/fanluo/miniconda3/lib/python3.7/site-packages') 
import en_core_web_md
nlp = en_core_web_md.load()       # load pretrained models 

#!pip install spacy-wordnet
import sys
sys.path.insert(-1, '/home/u32/fanluo/.local/lib/python3.5/site-packages') 
# import nltk
# dler = nltk.downloader.Downloader()
# dler._update_index()
# dler.download("omw")            # omw: Open Multilingual Wordnet 
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')

### synsets

In [17]:
doc = nlp('I would like to withdraw 5,000 euros') 
token = doc[4]

table = PrettyTable()  
# hypernyms: more abstract concepts
# hyponyms: more specific concepts
table.field_names = ["name", "definition", "examples", "lemmas", "hypernyms", "hyponyms"] 
table.align = "l"

synsets = token._.wordnet.synsets() 
for syn in synsets: 
    table.add_row([syn.name(), fill(syn.definition(), width=50), fill( ",".join(syn.examples()), width=30), fill(",".join(syn.lemma_names()), width=30), ",".join(hypernym.name() for hypernym in syn.hypernyms()), fill( ",".join(hypernym.name() for hypernym in syn.hyponyms()), width=30)])
print(table) 

+----------------+----------------------------------------------------+--------------------------------+--------------------------------+------------------+--------------------------------+
| name           | definition                                         | examples                       | lemmas                         | hypernyms        | hyponyms                       |
+----------------+----------------------------------------------------+--------------------------------+--------------------------------+------------------+--------------------------------+
| withdraw.v.01  | pull back or move away or backward                 | The enemy withdrew,The limo    | withdraw,retreat,pull_away,dra | travel.v.01      | back_up.v.02,fall_back.v.03,re |
|                |                                                    | pulled away from the curb      | w_back,recede,pull_back,retire |                  | trograde.v.03                  |
|                |                                

### lemmas

In [5]:
table = PrettyTable()  
table.field_names = ["name", "synset", "antonyms"] 
table.align = "l"

lemmas = token._.wordnet.lemmas() 

for lemma in lemmas: 
    table.add_row([lemma.name(), lemma.synset().name(), fill( ",".join(antonym.name() for antonym in lemma.antonyms()), width=50)])
print(table) 

+---------------------+----------------+----------+
| name                | synset         | antonyms |
+---------------------+----------------+----------+
| withdraw            | withdraw.v.01  |          |
| retreat             | withdraw.v.01  |          |
| pull_away           | withdraw.v.01  |          |
| draw_back           | withdraw.v.01  |          |
| recede              | withdraw.v.01  | advance  |
| pull_back           | withdraw.v.01  |          |
| retire              | withdraw.v.01  |          |
| move_back           | withdraw.v.01  |          |
| retire              | retire.v.02    |          |
| withdraw            | retire.v.02    |          |
| disengage           | disengage.v.01 | engage   |
| withdraw            | disengage.v.01 |          |
| recall              | recall.v.07    |          |
| call_in             | recall.v.07    |          |
| call_back           | recall.v.07    |          |
| withdraw            | recall.v.07    |          |
| swallow   

### domains

In [10]:
token._.wordnet.wordnet_domains()

['astronomy',
 'school',
 'telegraphy',
 'industry',
 'psychology',
 'ethnology',
 'ethnology',
 'administration',
 'school',
 'finance',
 'economy',
 'exchange',
 'banking',
 'commerce',
 'medicine',
 'ethnology',
 'university',
 'school',
 'buildings',
 'factotum',
 'agriculture',
 'mechanics',
 'gastronomy',
 'meteorology',
 'physics',
 'basketball',
 'anatomy',
 'skiing',
 'nautical',
 'engineering',
 'racing',
 'home',
 'drawing',
 'dentistry',
 'ethnology',
 'mathematics',
 'furniture',
 'animal_husbandry',
 'industry',
 'economy',
 'body_care',
 'chemistry',
 'medicine',
 'surgery',
 'vehicles',
 'transport',
 'atomic_physic',
 'archaeology',
 'hydraulics',
 'oceanography',
 'golf',
 'sculpture',
 'earth',
 'applied_science',
 'artisanship']

In [19]:
doc[0]._.wordnet.wordnet_domains()

['pharmacy',
 'chemistry',
 'physics',
 'applied_science',
 'photography',
 'number',
 'philosophy']

In [11]:
economy_domains = ['finance', 'banking']
enriched_sentence = []  

for token in doc:
    # We get those synsets within the desired domains
    synsets = token._.wordnet.wordnet_synsets_for_domain(economy_domains)
    if not synsets:
        enriched_sentence.append(token.text)
    else: 
        lemmas_for_synset = [lemma for s in synsets for lemma in s.lemma_names()] 
        enriched_sentence.append('({})'.format('|'.join(set(lemmas_for_synset))))

# Let's see our enriched sentence
print(' '.join(enriched_sentence))

I would (like|care|wish) to (withdraw|draw_off|draw|take_out) 5,000 euros
